#Perceptron - Pima Indians Diabetes Classification

##Prepare Packages and Google Collab

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, roc_auc_score

In [ ]:
#mount the google drive to access the data 
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


##Initialise Dataset

###Real World Dataset

In [2]:
#Load Dataset
data = pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#Remove classification column
variables = data.drop("Outcome", axis = 1)
#Create new data frame with classification column
classification = data["Outcome"]
#Normalize data
variables = MinMaxScaler().fit_transform(variables)
#Turn Data back into a dataframe
variables = pd.DataFrame(variables)
variables.head()

,0,1,2,3,4,5,6,7
0,0.352941,0.743719,0.590164,0.353535,0.000000,0.500745,0.234415,0.483333
1,0.058824,0.427136,0.540984,0.292929,0.000000,0.396423,0.116567,0.166667
2,0.470588,0.919598,0.524590,0.000000,0.000000,0.347243,0.253629,0.183333
3,0.058824,0.447236,0.540984,0.232323,0.111111,0.418778,0.038002,0.000000
4,0.000000,0.688442,0.327869,0.353535,0.198582,0.642325,0.943638,0.200000


In [4]:
classification.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [5]:
#Split dataset into test and training sets
train_X, test_X, train_Y, test_Y = train_test_split(variables, classification, test_size=0.2, random_state=0)
#Check the size of the training and testing sets
print(train_X.shape)
print(test_X.shape)
print(train_Y.shape)
print(test_Y.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [6]:
#Convert dataset into numpy arrays
train_X = np.array(train_X)
test_X = np.array(test_X)
train_Y = np.array(train_Y)
test_Y = np.array(test_Y)

###Toy Dataset

For the toy dataset the iris dataset was used with data some data preparation demonstrated by Jun Li in his week 10 demonstration (https://colab.research.google.com/drive/1sdyzxla7RjCCrRWmlHVUXnIRtfyK4gCs?usp=sharing). The goal of the dataset is to simplify the iris dataset into a binary dataset with only 100 data points in order for the Perceptron to be tested on an easy data set to validate the algorithm works as expected

In [7]:
#Load the iris dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
#Minimise the X dataset into 2 data attributes and reduce the dataset to only 100 samples
X = data['data'][:100, :2]
#Take the first 100 target samples to match with the X attributes
y = data['target'][:100]
#Split the data set into training and testing set with 33% going to the test side
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

##Design the Perceptron

In [8]:
class Perceptron:
  #Create the constructor for the Perceptron and initialise parameters
  def __init__(self, learning_rate = 0.1, epochs = 10):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.weights = None
    self.bias = None
  
  #create an activation function
  def activation_func(self, x):
    #if x is greater or equal to 0 return 1 otherwise return 0 meaning if it is activated or not
    return 1 if x >= self.bias else 0
  
  #create the weighted sum for the hypothesis space
  def weighted_sum(self, x):
    #transpose of weights times the data samples plus the bias
    return np.dot(x, self.weights) + self.bias

  #Fit the data to the model
  def fit(self, X, y):
    #Initialise features
    #make the weights initially all zero based on he number of features in the data
    self.weights = np.zeros(X.shape[1])
    #initialise the bias to be 0
    self.bias = 0
    #initialise the count to be 0
    count = 0
    #covert labelled data into a numpy array for processing
    y_array = np.array(y)

    while(count <= self.epochs):
      #initialise an array to append the current predicted values to
      predict_array = []
      for i, x in enumerate(X):
        #calculate predicted values
        #print(self.weights)
        #print(x)
        linear_calc = self.weighted_sum(x)
        predicted_value = self.activation_func(linear_calc)
        #append predictins for the current epoch into an array
        predict_array.append(predicted_value)

        # print("linear calc:", linear_calc)
        # print("real value:", y_array[i])
        # print("predicted_value:", predicted_value)
        
        #Update the wegihts and biases based on an update rule
        #Update rule stating that if the predicted value is false but the real value is true increase the weights and decrease the bias based on the learning rate
        if y_array[i] == 1 and predicted_value == 0:
          self.weights = self.weights + self.learning_rate * x
          self.bias = self.bias - self.learning_rate
        #Update rule stating that if the predicted value is true but the real value is false increase the bias and decrease the weights based on the learning rate
        elif y_array[i] == 0 and predicted_value == 1:
          self.weights = self.weights - self.learning_rate * x
          self.bias = self.bias + self.learning_rate
        
        # print("weights" , self.weights)
        # print("bias", self.bias)
      #calculate the accuracy of the current epoch 
      curr_accuracy = accuracy_score(y_array, predict_array)
      print("No. Epoch:", count, ",", "Accuracy:", curr_accuracy)
      #increase the count
      count += 1
  
  #create the predict function
  def predict(self, X):
    #initialise an array to hold the predicted results
    Y_prediction = []
    for x in X:
      #calculate the approximation
      linear_calc = self.weighted_sum(x)
      #determine whether the perceptron will be activated based on the calculated approximation
      prediction = self.activation_func(linear_calc)
      #append the predicted results to the array and return the array
      Y_prediction.append(prediction)
    return np.array(Y_prediction)
  



##Test the Perceptron on the Toy Dataset

In [9]:
#construct perceptron
perceptron = Perceptron()
#fit data to the perceptron
perceptron.fit(X_train, y_train)

No. Epoch: 0 , Accuracy: 0.6625
No. Epoch: 1 , Accuracy: 0.85
No. Epoch: 2 , Accuracy: 0.6875
No. Epoch: 3 , Accuracy: 0.8625
No. Epoch: 4 , Accuracy: 0.9
No. Epoch: 5 , Accuracy: 0.85
No. Epoch: 6 , Accuracy: 0.9
No. Epoch: 7 , Accuracy: 0.8375
No. Epoch: 8 , Accuracy: 0.9
No. Epoch: 9 , Accuracy: 0.9
No. Epoch: 10 , Accuracy: 0.9375


In [10]:
#run the perceptron's predict function on the test data
pred = perceptron.predict(X_test)
#show the predicted values and actual values
print("predictions", pred)
print("actual outcomes", y_test)
#display the accuracy of the model on the toy dataset
print("Accuracy:", accuracy_score(y_test, pred))

predictions [1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0]
actual outcomes [1 1 1 0 0 0 0 1 0 0 0 0 1 0 1 0 1 1 0 0]
Accuracy: 1.0


##Train the Perecptron on the Real-World Dataset

In [11]:
#construct perceptron with 100 epochs instead of 10
perceptron = Perceptron(0.1, 100)
#fit data to the perceptron
perceptron.fit(train_X, train_Y)

No. Epoch: 0 , Accuracy: 0.5651465798045603
No. Epoch: 1 , Accuracy: 0.5504885993485342
No. Epoch: 2 , Accuracy: 0.5456026058631922
No. Epoch: 3 , Accuracy: 0.5407166123778502
No. Epoch: 4 , Accuracy: 0.5439739413680782
No. Epoch: 5 , Accuracy: 0.5570032573289903
No. Epoch: 6 , Accuracy: 0.5456026058631922
No. Epoch: 7 , Accuracy: 0.5667752442996743
No. Epoch: 8 , Accuracy: 0.5618892508143323
No. Epoch: 9 , Accuracy: 0.5651465798045603
No. Epoch: 10 , Accuracy: 0.5456026058631922
No. Epoch: 11 , Accuracy: 0.5472312703583062
No. Epoch: 12 , Accuracy: 0.5521172638436482
No. Epoch: 13 , Accuracy: 0.5553745928338762
No. Epoch: 14 , Accuracy: 0.5570032573289903
No. Epoch: 15 , Accuracy: 0.5684039087947883
No. Epoch: 16 , Accuracy: 0.5732899022801303
No. Epoch: 17 , Accuracy: 0.5716612377850163
No. Epoch: 18 , Accuracy: 0.5586319218241043
No. Epoch: 19 , Accuracy: 0.5618892508143323
No. Epoch: 20 , Accuracy: 0.5602605863192183
No. Epoch: 21 , Accuracy: 0.5423452768729642
No. Epoch: 22 , Accu

##Evaluation of the Perceptron


In [12]:
#run the perceptron's predict function on the test data
pred = perceptron.predict(test_X)
#check the shape of the prediction array to ensure the prediction function produced the right format
pred.shape

(154,)

In [13]:
#compare the predicted results with the true results 
print("predictions", pred)
print("actual outcomes", test_Y)
print("confusion matrix:")
confusion_matrix(test_Y, pred)

predictions [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
actual outcomes [1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 1 0
 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0
 0 1 0 1 0 0]
confusion matrix:


array([[104,   3],
       [ 44,   3]], dtype=int64)

In [14]:
print("Accuracy:", accuracy_score(test_Y, pred))
print("Precision:", precision_score(test_Y, pred))
print("F1 score:", f1_score(test_Y, pred))
print("Area Under the Curve Score:", roc_auc_score(test_Y, pred))

Accuracy: 0.6948051948051948
Precision: 0.5
F1 score: 0.11320754716981131
Area Under the Curve Score: 0.5178962020282362
